In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
from data_loader.dataLoader import KaggleCSVLoader

PROJECT_ROOT = Path().resolve()
sys.path.append(str(PROJECT_ROOT))

## Fetching Data

In [2]:
# Loading Kaggle Data
DATASET_NAME = "dhrubangtalukdar/fortune-500-companies-stock-data"

loader = KaggleCSVLoader(
    dataset_name=DATASET_NAME,
    raw_data_dir='data/raw'
)

stock_data = loader.run() # stock_data: dict[str, pd.DataFrame]

CVS files already exist. Skipping download.
Loading NVR.csv
Loading Halliburton.csv
Loading Altria_Group.csv
Loading Paccar.csv
Loading Constellation_Energy.csv
Loading Cintas.csv
Loading Warner_Bros_Discovery.csv
Loading EMCOR_Group.csv
Loading Northern_Trust.csv
Loading Gilead_Sciences.csv
Loading Mastercard.csv
Loading Thermo_Fisher_Scientific.csv
Loading Devon_Energy.csv
Loading United_Rentals.csv
Loading Thor_Industries.csv
Loading CBRE_Group.csv
Loading Genuine_Parts.csv
Loading Voya_Financial.csv
Loading Unum_Group.csv
Loading Carrier_Global.csv
Loading Fox.csv
Loading Corebridge_Financial.csv
Loading AutoZone.csv
Loading Huntington_Ingalls_Industries.csv
Loading Charter_Communications.csv
Loading Micron_Technology.csv
Loading Ross_Stores.csv
Loading Elevance_Health.csv
Loading Newmont.csv
Loading Lincoln_National.csv
Loading HCA_Healthcare.csv
Loading PepsiCo.csv
Loading Global_Payments.csv
Loading Alaska_Air_Group.csv
Loading Starbucks.csv
Loading Lithia_Motors.csv
Loading Afl

## Data Cleansing

In [3]:
# Required columns that have to be present in each stock data
cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Adjusted_Close', 'Volume', 'Ticker']

# Checking ticker columns and datetime column dtypes
valid_data = {}
invalid_tickers = []

for ticker, df in stock_data.items():
    # Check the missing columns
    if (not set(cols).issubset(df.columns)):
        invalid_tickers.append((ticker, 'Missing columns'))
        continue
    if not np.issubdtype(df['Date'].dtype, np.datetime64):
        invalid_tickers.append((ticker, 'Date not datetime'))
        continue

    valid_data[ticker] = df

print(f"Valid tickers: {len(valid_data)} \t Invalid tickers: {len(invalid_tickers)}")

# Remove duplicates & sort values based on date
for ticker, df in valid_data.items():
    df = (
        df
        .drop_duplicates(subset=['Date'])
        .sort_values('Date')
        .reset_index(drop=True)
    )

    valid_data[ticker] = df

### Data manipulation
# Enforcing data dtypes
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
enforced_data = {}

for ticker, df in valid_data.items():
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors = 'coerce')

    enforced_data[ticker] = df

# Handling missing data
clean_data = {}
min_length_data = {}
min_length = 252 # Nb. of trading days in a year

for ticker, df in enforced_data.items():
    price_cols = ['Open', 'High', 'Low', 'Close']
    df[price_cols] = df[price_cols].ffill()
    df['Volume'] = df['Volume'].fillna(0)
    
    # dropping NAs at the start of series
    df = df.dropna()

    if len(df) < min_length:
        min_length_data[ticker] = df
        continue

    clean_data[ticker] = df

print(f"Nb. of tickers with insufficient history {len(min_length_data)}")

# Sanity on logical expressions
def sanityCheck(df):
    return(
        (df['High'] >= df['Low']).all()
        and (df['Close'] > 0).all()
        and (df['Volume'] >= 0).all()
    )

clean_data = {
    t: df for t, df in clean_data.items() if sanityCheck(df)
}

dirty_data = {
    t: df for t, df in clean_data.items() if not(sanityCheck(df))
}
print(f"Clean tickers: {len(clean_data)} \t Dirty tickers: {len(dirty_data)}")

# Concatenating the tickers in the clean_data dictionary
clean_data_df = pd.concat(
    clean_data.values(),
    ignore_index = True
)

clean_data_df.groupby('Ticker').size().sort_values()

Valid tickers: 461 	 Invalid tickers: 0
Nb. of tickers with insufficient history 1
Clean tickers: 460 	 Dirty tickers: 0


Ticker
Ingram_Micro_Holding             296
GE_Vernova                       442
Solventum                        443
BrightSpring_Health_Services     484
KLA                              572
                                ... 
CHS                             2784
Kellanova                       2785
QVC_Group                       2795
Air_Products_and_Chemicals      2795
Seaboard                        2795
Length: 460, dtype: int64

In [4]:
# Saving the processed data to dedicated folder
processed_dir = Path("data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

clean_data_df.to_csv(
    processed_dir / "fortune_500_cleaned.csv",
    index = False
)

for ticker, df in clean_data.items():
    df.to_csv(
        processed_dir / f"{ticker}_cleaned.csv",
        index = False
    )